In [20]:
import pandas as pd
import numpy as np
import random
import os
import keras
from tensorflow.keras import models
from tensorflow.keras.models import load_model
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scipy.ndimage import rotate
np.random.seed(42)

In [25]:
dd = pd.read_csv("train.csv")
dd.head()

M = int(0.8 * len(dd))

dd2 = dd[['x', 'y', 'z', 'Vx', 'Vy', 'Vz', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']]
dd2 = dd2.sample(frac=1).reset_index(drop=True)

dd2.head()

train_input = dd2.iloc[0:M][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']].values
train_output = dd2.iloc[0:M][['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].values

test_input = dd2.iloc[M:][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']].values
test_output = dd2.iloc[M:][['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].values

In [17]:
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim = 6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal'))
    model.compile(loss = 'mean_squared_error', optimizer='adam')
    return model

In [33]:
model = create_model()
model.summary()
history = model.fit(train_input, train_output, epochs=3, batch_size=32)
model.save('model2.h5') 

In [21]:
model = load_model('model1.h5')
results = model.evaluate(test_input, test_output)
results

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
129983/129983 [==============================] - 2s 12us/sample - loss: 22808250.0360s - loss: 22812499.29


22808250.035950854

In [22]:
error_train = np.mean((train_input-train_output)**2)
error_test = np.mean((test_input-test_output)**2)

print((error_test-results)/error_test * 100)

2.1816759696605765


In [109]:
type(ids)

pandas.core.series.Series

In [27]:
FINAL_TEST_RAW = pd.read_csv("test.csv")
FINAL_TEST = FINAL_TEST_RAW[['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']].values
ids = FINAL_TEST_RAW['id']

RESULTS = model.predict(FINAL_TEST)

In [30]:
df = pd.DataFrame(data=RESULTS,
          index=ids,
          columns=['x', 'y', 'z', 'Vx', 'Vy', 'Vz'])
df.head()

,x,y,z,Vx,Vy,Vz
id,,,,,,
3927,-12145.327148,-13780.466797,5500.929199,1.137367,5.431465,11.036842
3928,-6825.518555,-13862.408203,6297.592285,0.908180,5.242375,12.167385
3929,-289.557068,-12548.290039,6389.449219,0.764357,4.133708,13.651603
3930,6182.983398,-8777.461914,5240.117188,0.455378,2.266691,12.360672
3931,10436.258789,-2116.989990,2496.424805,0.096635,0.364832,10.978507


In [32]:
df.to_csv('submission.csv')

In [34]:
def score(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    smape =  np.mean(np.abs((satellite_predicted_values - satellite_true_values) 
        / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))
    return 100 * (1 - smape)

In [41]:
score(train_input, train_output)

81.28598489507178

In [49]:
raw = dd[dd['sat_id'] <= 50][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']].values
real = dd[dd['sat_id'] <= 50][['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].values
improved = pd.read_csv('pred.csv')

print(score(raw, real))
print(score(improved, real))

,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133
1,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
2,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768
3,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306
4,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237
